In [ ]:
import os
os.listdir("../input/cifar-10")

In [ ]:
import pandas as pd
trainLabels = pd.read_csv("../input/cifar-10/trainLabels.csv")
trainLabels

In [ ]:
!pip install py7zr

In [ ]:
!python -m py7zr x ../input/cifar-10/train.7z #takes 10 oof minutes of CPU to decompress this file. Big OOF

In [ ]:
#!python -m py7zr x ../input/cifar-10/test.7z # No sir, I shan't decompress thou

In [ ]:
import pandas as pd
import glob
import numpy as np
from PIL import Image
y = pd.read_csv('../input/cifar-10/trainLabels.csv')
filelist = glob.glob("../working/train/*")
filelist = sorted(filelist,key= lambda x:int(x.split("/")[-1].split(".")[0]))
X = np.array([np.array(Image.open(fname)) for fname in filelist])
print(f"X.shape : {X.shape} y.shape : {y.shape}")

In [ ]:
# got X and y at this point, now some preprocessing
import tensorflow
y = y['label'].values

X = X.astype('float32')

X = X/255.0

temp = y
categories = list(pd.CategoricalIndex(temp).categories)
mapping = dict()
for i in range(len(categories)):
    mapping[categories[i]] = i
for i in range(len(temp)):
    temp[i] = mapping[temp[i]]
y = tensorflow.keras.utils.to_categorical(temp)

In [ ]:
print(f"X.shape : {X.shape} , y.shape : {y.shape}")

In [ ]:
#preprocessing finally completed, now creating a basic model with keras

from tensorflow.keras import Sequential
from tensorflow.keras.layers import *


model = Sequential()
model.add(Conv2D(24,kernel_size=(3,3),padding='same',activation='relu',input_shape=(32,32,3)))
model.add(BatchNormalization())
model.add(Conv2D(24,kernel_size=(3,3),padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(32,kernel_size=5,strides=2,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64,kernel_size=5,strides=2,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Conv2D(128,kernel_size=4,activation='relu'))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(10,activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()




In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
modelMP = Sequential()
modelMP.add(Conv2D(24,kernel_size=(3,3),padding='same',activation='relu',input_shape=(32,32,3)))
modelMP.add(BatchNormalization())
modelMP.add(Conv2D(24,kernel_size=(3,3),padding='same',activation='relu'))
modelMP.add(BatchNormalization())
modelMP.add(Conv2D(32,kernel_size=5,strides=2,padding='same',activation='relu'))
modelMP.add(BatchNormalization())
modelMP.add(Dropout(0.4))
modelMP.add(Conv2D(64,(3,3),activation='relu'))
modelMP.add(BatchNormalization())
modelMP.add(Conv2D(64,kernel_size=5,strides=2,padding='same',activation='relu'))
modelMP.add(BatchNormalization())
modelMP.add(Dropout(0.4))
modelMP.add(Conv2D(128,kernel_size=4,activation='relu'))
modelMP.add(BatchNormalization())
modelMP.add(Conv2D(256,kernel_size=3,activation='relu'))
modelMP.add(BatchNormalization())
modelMP.add(Conv2D(512,kernel_size=2,activation='relu'))
modelMP.add(BatchNormalization())
modelMP.add(Flatten())
modelMP.add(Dropout(0.4))
modelMP.add(Dense(10,activation='softmax'))
modelMP.summary()

In [ ]:
modelMP.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# data augmentation
# starting Data Augmentation now

from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
            rotation_range=10,
            zoom_range=0.1,
            width_shift_range=0.1,
            height_shift_range=0.1
            )
datagen.fit(X)

In [ ]:
# for validation testing, base model : with 10+40 epochs reached 84% accuracy on the private leaderboard
from sklearn.model_selection import train_test_split

X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=1)
model.fit(datagen.flow(X_train,y_train),batch_size=128,epochs=10,verbose=1,validation_data=(X_val,y_val))

In [ ]:
# for validation testing
from sklearn.model_selection import train_test_split

X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=1)
modelMP.fit(datagen.flow(X_train,y_train),batch_size=128,epochs=10,verbose=1,validation_data=(X_val,y_val))

In [ ]:
# for actual
modelMP.fit(datagen.flow(X,y),batch_size=128,epochs=40,verbose=1)

In [ ]:
# decompressing the testing data
!python -m py7zr x ../input/cifar-10/test.7z # No sir, I shan't decompress thou

In [ ]:
# taking input
testfilelist = glob.glob("../working/test/*")
testfilelist = sorted(testfilelist,key= lambda x:int(x.split("/")[-1].split(".")[0]))
X_test = np.array([np.array(Image.open(fname)) for fname in testfilelist])

In [ ]:
# preprocessing the testing dataset
X_test = X_test.astype('float32')
X_test = X_test/255.0

In [ ]:
# reverse mapping of labels vs indices
revMapping = {}
for x in mapping.keys():
    revMapping[mapping[x]] =  x
revMapping

In [ ]:
# delete everything that is of no use now
del X
del y


In [ ]:
# predicting on the testing dataset
results = modelMP.predict(X_test)
# delete X_test now, we only need the results now
del X_test
results = results.argmax(axis=1)
ans = []
for i in range(len(results)):
    ans.append(revMapping[results[i]])
ans

In [ ]:
submission = pd.concat([pd.Series(range(1,len(ans)+1),name='id').astype('int32'),pd.Series(ans,name='label')],axis=1)
submission.to_csv('submission4.csv',index=False)

In [ ]:
# testing

import numpy as np
revMapping = {0: 'airplane',
 1: 'automobile',
 2: 'bird',
 3: 'cat',
 4: 'deer',
 5: 'dog',
 6: 'frog',
 7: 'horse',
 8: 'ship',
 9: 'truck'}


In [ ]:
arr = np.zeros(10)
for i in range(10):
    arr[i] = i 
arr

In [ ]:
charray = np.array([revMapping[x] for x in arr])
charray

In [ ]:
int("21")==float(21)

In [ ]:
2**++-3-3

In [ ]:
2+++++3
2++++++3
2+++++++3

In [ ]:
import os
os.chdir(r'../working')
from IPython.display import FileLink
FileLink(r'submission4.csv')